In [11]:
import os
from pypdf import PdfReader
import layoutparser as lp
import fitz
from PIL import Image
import io
from langchain_community.document_loaders import PyPDFLoader

In [9]:
def render_page_as_image(pdf_document, page_number):
    page = pdf_document[page_number]
    pix = page.get_pixmap()
    img_bytes = pix.tobytes("png")  # Export as PNG bytes
    img = Image.open(io.BytesIO(img_bytes))  # Convert bytes to PIL Image
    return img

In [3]:
pdf_root = "/datasets/ureason/PDF_Manuals_OEM_LLM"
pdf_file = os.path.join(pdf_root, "Fischer_657_Manual.pdf")
print(pdf_file)

/datasets/ureason/PDF_Manuals_OEM_LLM/Fischer_657_Manual.pdf


### Load the model

In [16]:
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'
# !pip install -U 'git+https://github.com/facebookresearch/detectron2.git@ff53992b1985b63bd3262b5a36167098e3dada02'
# !ls /root/.torch/iopath_cache/s/dgy9c10wykk4lq4
# !mv /root/.torch/iopath_cache/s/dgy9c10wykk4lq4/model_final.pth?dl=1 /root/.torch/iopath_cache/s/dgy9c10wykk4lq4/model_final.pth
# !rm /root/.torch/iopath_cache/s/dgy9c10wykk4lq4/model_final.pth?dl=1
!ls /root/.torch/iopath_cache/s/dgy9c10wykk4lq4 -a
# !echo $LD_PRELOAD
# !apt-get install g++ -y

 .   ..   model_final.pth  'model_final.pth?dl=1'  'model_final.pth?dl=1.lock'


In [17]:
from layoutparser.models import Detectron2LayoutModel, detectron2
import requests
import copy
import os


class ExtractLayout(Detectron2LayoutModel):

    def __init__(self,
                 config_path: str = 'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
                 *args,
                 **kwargs
                 ):
        """
        The following modified __init__ is to solve this issue:
                 https://github.com/Layout-Parser/layout-parser/issues/168

        :param config_path: A path to the config file
        """

        config_path_split = config_path.split('/')
        dataset_name = config_path_split[-3]
        model_name = config_path_split[-2]

        model_url = Detectron2LayoutModel.MODEL_CATALOG[dataset_name][model_name]
        config_url = detectron2.catalog.CONFIG_CATALOG[dataset_name][model_name]

        if 'model' not in os.listdir():
            os.mkdir('model')

        config_file_path, model_file_path = None, None

        for url in [model_url, config_url]:
            filename = url.split('/')[-1].split('?')[0]
            save_to_path = f"model/" + filename
            if 'config' in filename:
                config_file_path = copy.deepcopy(save_to_path)
            if 'model_final' in filename:
                model_file_path = copy.deepcopy(save_to_path)
            if filename in os.listdir("model"):
                continue
            r = requests.get(url, stream=True, headers={'user-agent': 'Wget/1.16 (linux-gnu)'})

            with open(save_to_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=4096):
                    if chunk:
                        f.write(chunk)

        super().__init__(
            config_path=config_file_path,
            model_path=model_file_path,
            *args,
            **kwargs
        )

In [18]:
import layoutparser as lp
model = ExtractLayout('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
                        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                        label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})

/opt/conda/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

In [24]:
img = Image.open("/datasets/ureason/image.png")

In [43]:
page_number = 18
pdf_doc = fitz.open(pdf_file)
# .pages[page_number-1]
page_image = render_page_as_image(pdf_doc, page_number)

In [1]:
layout = model.detect(page_image)
lp.draw_box(page_image, layout, box_width=3)

NameError: name 'model' is not defined

#########

In [4]:
loader = PyPDFLoader(pdf_file)
reader = PdfReader(pdf_file)
pages = loader.load()

In [75]:
reader.xref

{0: {290: 16,
  291: 2493,
  292: 2578,
  293: 3263,
  294: 3390,
  295: 3516,
  296: 3641,
  297: 3766,
  298: 3891,
  299: 4016,
  300: 4141,
  301: 4267,
  302: 4393,
  303: 4519,
  304: 4645,
  305: 4771,
  306: 4896,
  307: 5021,
  308: 5147,
  309: 5272,
  310: 5398,
  311: 5525,
  312: 5652,
  313: 5779,
  314: 5906,
  315: 6033,
  316: 6160,
  317: 6287,
  318: 6414,
  319: 6545,
  320: 6675,
  321: 6806,
  322: 6936,
  323: 7067,
  324: 7194,
  325: 7325,
  326: 7455,
  327: 7586,
  328: 7717,
  329: 7848,
  330: 7979,
  331: 8110,
  332: 8241,
  333: 8372,
  334: 8503,
  335: 8634,
  336: 8764,
  337: 9385,
  338: 9985,
  339: 10035,
  340: 10149,
  341: 10611,
  342: 17777,
  343: 18361,
  344: 18744,
  345: 19058,
  346: 24931,
  347: 25361,
  348: 25726,
  349: 28287,
  350: 28896,
  351: 29234,
  352: 35086,
  353: 35554,
  354: 35919,
  355: 36453,
  356: 40959,
  357: 43585,
  358: 1676,
  1: 55674,
  2: 55958,
  3: 56086,
  4: 56215,
  5: 56341,
  6: 56468,
  7: 56593,